In [8]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.callbacks import ModelCheckpoint

원본 이미지 로드 및 전처리

In [107]:
img = cv2.imread('baby.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img.shape

(2876, 2924, 3)

데이터 전처리

In [4]:
# 이미지 리사이징

def prepare_images(path, factor):
    
    for file in os.listdir(path):
        img = cv2.imread(path + '/' + file)

        h, w, c = img.shape
        new_height = int(h / factor)
        new_width = int(w / factor)
        
        img = cv2.resize(img, (new_width, new_height), interpolation = cv2.INTER_LINEAR)
        img = cv2.resize(img, (w, h), interpolation = cv2.INTER_LINEAR)

        print('Saving {}'.format(file))
        cv2.imwrite('images/{}'.format(file), img)

In [5]:
prepare_images('source', 2)

Saving 0.png
Saving 1.png
Saving 10.png
Saving 11.png
Saving 12.png
Saving 13.png
Saving 14.png
Saving 15.png
Saving 16.png
Saving 17.png
Saving 18.png
Saving 19.png
Saving 2.png
Saving 20.png
Saving 21.png
Saving 22.png
Saving 23.png
Saving 24.png
Saving 25.png
Saving 26.png
Saving 27.png
Saving 28.png
Saving 29.png
Saving 3.png
Saving 30.png
Saving 31.png
Saving 32.png
Saving 33.png
Saving 34.png
Saving 35.png
Saving 36.png
Saving 37.png
Saving 38.png
Saving 39.png
Saving 4.png
Saving 40.png
Saving 41.png
Saving 42.png
Saving 43.png
Saving 44.png
Saving 45.png
Saving 46.png
Saving 47.png
Saving 48.png
Saving 49.png
Saving 5.png
Saving 50.png
Saving 51.png
Saving 52.png
Saving 53.png
Saving 54.png
Saving 55.png
Saving 56.png
Saving 57.png
Saving 58.png
Saving 59.png
Saving 6.png
Saving 60.png
Saving 61.png
Saving 62.png
Saving 63.png
Saving 64.png
Saving 65.png
Saving 66.png
Saving 67.png
Saving 68.png
Saving 69.png
Saving 7.png
Saving 70.png
Saving 71.png
Saving 72.png
Saving 73.png


원본과 비교

In [2]:
def modcrop(img, scale):
    tmpsz = img.shape
    sz = tmpsz[0:2]
    sz = sz - np.mod(sz, scale)
    img = img[0:sz[0], 0:sz[1]]
    return img

def shave(image, border):
    img = image[border:-border, border:-border]
    return img

모델링

In [7]:
def model():
    
    # define model type
    SRCNN = Sequential()
    
    # add model layers
    SRCNN.add(Conv2D(filters=128, kernel_size=(9, 9), kernel_initializer='glorot_uniform',
                     activation='relu', padding='valid', use_bias=True, input_shape=(None, None, 1)))
    SRCNN.add(Conv2D(filters=64, kernel_size=(3, 3), kernel_initializer='glorot_uniform',
                     activation='relu', padding='same', use_bias=True))
    SRCNN.add(Conv2D(filters=1, kernel_size=(5,5), kernel_initializer='glorot_uniform',
                     activation='linear', padding='valid', use_bias=True))
    
    # compile model
    SRCNN.compile(optimizer='adam', loss='mse', metrics=['mse'])
    
    return SRCNN


In [ ]:
def train():
    srcnn_model = model()
    print(srcnn_model.summary())
    data, label = pd.read_training_data("./train.h5")
    val_data, val_label = pd.read_training_data("./test.h5")

    checkpoint = ModelCheckpoint("SRCNN_check.h5", monitor='val_loss', verbose=1, save_best_only=True,
                                 save_weights_only=False, mode='min')
    callbacks_list = [checkpoint]

    srcnn_model.fit(data, label, batch_size=128, validation_data=(val_data, val_label),
                    callbacks=callbacks_list, shuffle=True, nb_epoch=200, verbose=0)

예측

In [119]:
def predict(image_path):
    
    # load the SRCNN model with weights
    srcnn = model()
    srcnn.load_weights('3051crop_weight_200.h5')
    
    # load the degraded and reference images
    ref = cv2.imread('baby.jpg')
    degraded = cv2.imread(image_path)

    
    # preprocess the image with madcrop
    ref = modcrop(ref, 10)
    degraded = modcrop(degraded, 10)
    
    # convert the image to VCrCb - (srccn trained on Y channel)
    temp = cv2.cvtColor(degraded, cv2.COLOR_BGR2YCrCb)
    
    # create image slice and normalize
    Y = np.zeros((1, temp.shape[0], temp.shape[1], 1), dtype=float)
    Y[0, :, :, 0] = temp[:, :, 0].astype(float) / 255
    
    # perform super-resolution with srcnn
    pre = srcnn.predict(Y, batch_size=100)
    
    # post-process output
    pre *= 255
    pre[pre[:] > 255] = 255
    pre[pre[:] < 0] = 0
    pre = pre.astype(np.uint8)
    
    # copy Y channel back to image and convert to BGR
    temp = shave(temp, 6)
    temp[:, :, 0] = pre[0, :, :, 0]
    output = cv2.cvtColor(temp, cv2.COLOR_YCrCb2BGR) 
    
    # remove border from reference and degraded image
    ref = shave(ref.astype(np.uint8), 6)
    degraded = shave(degraded.astype(np.uint8), 6)
    
    # return images and scores
    return ref, degraded, output
    

In [120]:
ref, degraded, output = predict('downgraded_baby.jpg')

# display images as subplots
fig, axs = plt.subplots(1, 3, figsize=(20,8))
axs[0].imshow(cv2.cvtColor(ref, cv2.COLOR_BGR2RGB))
axs[0].set_title("Original")
axs[1].imshow(cv2.cvtColor(degraded, cv2.COLOR_BGR2RGB))
axs[1].set_title("Degraded")
axs[2].imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
axs[2].set_title("SRCNN")

# remove the x and y tick marks
for ax in axs:
    ax.set_xticks([])
    ax.set_yticks([])

ResourceExhaustedError:  OOM when allocating tensor with shape[1,64,2862,2912] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential_7/conv2d_19/Relu
 (defined at C:\ProgramData\Anaconda3\lib\site-packages\keras\backend.py:4867)
]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_predict_function_953]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_7/conv2d_19/Relu:
In[0] sequential_7/conv2d_19/BiasAdd (defined at C:\ProgramData\Anaconda3\lib\site-packages\keras\layers\convolutional.py:264)

Operation defined at: (most recent call last)
>>>   File "C:\ProgramData\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "C:\ProgramData\Anaconda3